In [1]:
import numpy as np
import pandas as pd

In [2]:
data = (pd.read_csv('2in_complex.txt', names = ['x', 'y', 'z'], delimiter = '\t'))

dataX = np.array(data.drop(columns = 'z'))

dataY = np.array(data['z'])

In [3]:
def initWeights(n_layers):
    W = list()
    for i in range(1, len(n_layers)):
        W.append(np.random.randn(n_layers[i], n_layers[i - 1]) * 0.01)
    return W

def initBiases(n_layers):
    B = list()
    for i in range(1, len(n_layers)):
        B.append(np.zeros(n_layers[i]))
    return B

def initNetworkX(n_layers):
    X = list()
    for i in range(len(n_layers)):
        X.append(np.zeros(n_layers[i]))
    return X

In [4]:
def null(z):
    return 0

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanH(z):
    return np.tanh(z)

def cosine(z):
    return np.cos(z)

def gaussian(z):
    return np.exp(-(np.square(z) / 2))

vNull = np.vectorize(null)

In [5]:
def forwardPass(X, W, B, act):
    
    if act == 'sigmoid':
        for i in range(len(X)):
            if i != (len(X) - 1):
                XWB = np.dot(W[i], X[i]) + B[i]
                X[i + 1] = sigmoid(XWB)
            else:
                return X[-1]
                
    elif act == 'tanh':
        for i in range(len(X)):
            if i != (len(X) - 1):
                XWB = np.dot(W[i], X[i]) + B[i]
                X[i + 1] = tanH(XWB)
            else:
                return X[-1]
                
    elif act == 'cos':
        for i in range(len(X)):
            if i != (len(X) - 1):
                XWB = np.dot(W[i], X[i]) + B[i]
                X[i + 1] = cosine(XWB)
            else:
                return X[-1]

    elif act == 'gaussian':
        for i in range(len(X)):
            if i != (len(X) - 1):
                XWB = np.dot(W[i], X[i]) + B[i]
                X[i + 1] = gaussian(XWB)
            else:
                return X[-1]

In [6]:
def MSE(y_pred, y_true):
    return np.square(np.subtract(y_true,y_pred)).mean() 

In [7]:
def oneEpochLoss(W, B, data, y_true, act):
    val_pred = list()
    X = initNetworkX(n_layers)
    for row in data:
        X = initNetworkX(n_layers)
        X[0] = row
        val_pred.append(forwardPass(X, W, B, act))
    return MSE(np.array(val_pred), y_true)

In [8]:
def initvWeights(n_layers):
    W = list()
    for i in range(1, len(n_layers)):
        W.append(np.random.uniform(low=-1, high=1, size=(n_layers[i], n_layers[i - 1])))
    return W

In [9]:
def initvBiases(n_layers):
    W = list()
    for i in range(1, len(n_layers)):
        W.append(np.random.uniform(low=-10, high=10, size=(n_layers[i])))
    return W

In [10]:
def makeParticle(n_layers):
    W = initvWeights(n_layers)
    B = initBiases(n_layers)
    vW = initvWeights(n_layers)
    vB = initvBiases(n_layers)
    return W, B, vW, vB

In [11]:
def makeSwarm(nP, n_layers):#nSS, n_layers):
    S = list()
    for i in range(nP):
        S.append(makeParticle(n_layers))#nSS, n_layers))
    return S

In [12]:
def swarmFitness(S, act):
    E = list()
    for p in S:
        E.append(oneEpochLoss(p[0], p[1], dataX, dataY, act))
    return E

In [13]:
def bestParticle(E):
    return E.index(min(E))

In [14]:
def bcd(beta, gamma, delta, shape):
    b = np.random.uniform(low=0, high=beta, size=shape)
    c = np.random.uniform(low=0, high=gamma, size=shape)
    d = np.random.uniform(low=0, high=delta, size=shape)
    return b, c, d

In [15]:
def pickInformants(E, S, informants):
    I = list()
    eI = list()
    idx = list()
    
    for informers in range(informants):
        idx.append(np.random.choice(range(len(S))))

    for ids in idx:
        I.append(S[ids])
        eI.append(E[ids])
    
    return eI, I

In [16]:
def mainLoop(maxIter, cutoff, activation, informants, E_curr, E_best, S_curr, S_best):
    
    for iterations in range(maxIter):
        
        E_curr = swarmFitness(S_curr, activation)
        
        for i in range(len(E_curr)):
            if E_curr[i] < E_best[i]:
                E_best[i] = E_curr[i]
                S_best[i] = S_curr[i]

        if min(E_best) == error_cutoff:
            return E_best, S_best, E_curr, S_curr

        for p in range(len(S_curr)):
            bestMe = S_best[p]

            bestAll = S_best[bestParticle(E_best)]

            eI, I = pickInformants(E_best, S_best, informants)
            eIdx = bestParticle(eI)
            bestI = I[eIdx]

            for w in range(len(S_curr[p][0])):
                b, c, d = bcd(beta, gamma, delta, S_curr[p][2][w].shape)
                avi = alpha * S_curr[p][2][w]
                bx = b * ((bestMe[0][w] - S_curr[p][0][w]))
                cx = c * (bestI[0][w] - S_curr[p][0][w])
                dx = d * (bestAll[0][w] - S_curr[p][0][w])
                S_curr[p][2][w] = avi + bx + cx + dx
                S_curr[p][0][w] = S_curr[p][0][w] + (epsilon * S_curr[p][2][w])
                
            for b in range(len(S_curr[p][1])):
                b2, c, d = bcd(beta, gamma, delta, S_curr[p][3][b].shape)
                avi = alpha * S_curr[p][3][b]
                bx = b2 * ((bestMe[1][b] - S_curr[p][1][b]))
                cx = c * (bestI[1][b] - S_curr[p][1][b])
                dx = d * (bestAll[1][b] - S_curr[p][1][b])
                S_curr[p][3][b] = avi + bx + cx + dx
                S_curr[p][1][b] = S_curr[p][1][b] + (epsilon * S_curr[p][3][b])
                
    return E_best, S_best, E_curr, S_curr

In [17]:
error_cutoff = 0.002
maxIter = 250
i_layers = [len(data.drop(columns = 'z').columns)]
o_layers = [1]
particlePossibilities = [i for i in range(10, 101)]
rndHParams = list()
rndOut = list()
rndE_best = list()
rndE_curr = list()

In [18]:
for i in range(75):
    print(i)
    n_layers = i_layers + [np.random.choice([2, 3, 4, 5, 6]) for i in range(1, np.random.choice([2, 3, 4, 5]))] + o_layers
    beta = np.random.uniform(low = 0, high = 1.33)
    gamma = np.random.uniform(low = 0, high = 1.33)
    delta = np.random.uniform(low = 0, high = 1.33)
    alpha = np.random.uniform(low = 0.2, high = 2)
    numParticles = np.random.choice(particlePossibilities)
    informants = np.random.choice([i for i in range(10, (numParticles+1))])
    epsilon = np.random.uniform(low = 0.2, high = 2)
    act = np.random.choice(['sigmoid', 'gaussian', 'tanh', 'cos'])
    params = [n_layers, beta, gamma, delta, alpha, numParticles, informants, epsilon, act]
    rndHParams.append(params)
    S_curr = makeSwarm(numParticles, n_layers)
    S_best = S_curr
    E_best = swarmFitness(S_best, act)
    E_curr = E_best
    E_best, S_best, E_curr, S_curr = mainLoop(maxIter, error_cutoff, act, informants, E_curr, E_best, S_curr, S_best)
    rndE_best.append(E_best[bestParticle(E_best)])
    rndE_curr.append(E_curr[bestParticle(E_curr)])
    rndOut.append([E_best, S_best, E_curr, S_curr])

0
1
2
3
4
5
6
7
8


<ipython-input-4-5dc441157d47>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74


In [54]:
rndE_best[bestParticle(rndE_best)]

0.1214765364000266

In [21]:
rndHParams[bestParticle(rndE_best)]

[[2, 5, 2, 4, 2, 1],
 0.20132655128594262,
 1.1099957893069692,
 0.0657399286443541,
 0.2796924161325599,
 48,
 44,
 1.69325977230263,
 'tanh']